In [2]:
import process as pr
import model as md
import pandas as pd
import forest as fr
%load_ext autoreload
%autoreload 2

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats

In [ ]:
# the raw training data
raw = pr.get_raw('train')
# the processed training data
train_p = pr.get_post('train')
# the random forest
rf = fr.get_forest(train_p)
# the processed test data
test_p = pr.get_post('test')
# predicting the probabilities and aggregating the data
final = md.prep_for_submission(rf, test_p)
# saving the file to csv for upload

In [62]:
X = train_p.drop('TripType', axis=1)
y = train_p.TripType # equivalent y = train_p.loc[:, 'TripType']

In [67]:
# without some adjustment, k nearest neighbors won't work well -- the classes are highly skewed
perc = train_p.groupby('TripType').size() / train_p.shape[0] * 100.0
pd.DataFrame(perc, columns=['percDist']).sort_values(by='percDist')

,percDist
TripType,
14,0.005409
23,0.050228
4,0.139246
19,0.183601
31,0.272775
29,0.325321
12,0.325784
26,0.387448
28,0.411712


In [60]:
from sklearn.cross_validation import train_test_split

In [63]:
train_x, test_x, labels_x, labels_y = train_test_split(X, y)

In [72]:
train_p_only_agg = train_p[['TripType', ]]

,TripType,VisitNumber,Upc,ScanCount,FinelineNumber,Friday,Monday,Saturday,Sunday,Thursday,...,containsReturn,mostFreqFineLine,totalItemsBought,totalDistinctItemsBought,itemDist,numTransactions,totalReturns,numUniqueFinelines,avePurchaseSize,medianPurchaseSize
0,999,5,68113152929,-1,1000,1,0,0,0,0,...,1,-1,0,0,0.000000,1,1,1,-1.000000,-1
1,30,7,60538815980,1,8931,1,0,0,0,0,...,0,1,2,2,0.000000,2,0,2,1.000000,1
2,30,7,7410811099,1,4504,1,0,0,0,0,...,0,1,2,2,0.000000,2,0,2,1.000000,1
3,26,8,2238403510,2,3565,1,0,0,0,0,...,1,1,30,21,0.882512,23,2,17,1.217391,1
4,26,8,2006613744,2,1017,1,0,0,0,0,...,1,1,30,21,0.882512,23,2,17,1.217391,1


In [79]:
keep_col = ['TripType', 'containsReturn', 'totalItemsBought', 'totalDistinctItemsBought', 'itemDist', 'numTransactions', 'totalReturns', 'numUniqueFinelines', 'avePurchaseSize', 'medianPurchaseSize']
train_only_agg = train_p[keep_col]

In [242]:
train = pr.get_post('train')

In [243]:
rf = fr.get_forest(train)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   16.7s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:  1.1min
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:  2.4min
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  2.7min finished


In [244]:
rf.score

<bound method RandomForestClassifier.score of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=True,
            warm_start=False)>

In [198]:
test = pr.get_post('test')

In [245]:
final7 = md.prep_for_submission(rf=rf, test=test)
final7.shape

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    2.7s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   11.7s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:   26.1s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   29.0s finished


(95674, 39)

In [246]:
final7.to_csv('f7777.csv', index=False)